In [ ]:
!pip install beir

In [2]:
from beir import util, LoggingHandler

import logging
import pathlib, os

from beir.datasets.data_loader import GenericDataLoader

from sentence_transformers import SentenceTransformer

from sentence_transformers import util as sentence_transformers_util

from tqdm.notebook import tqdm

import numpy as np

import random

from typing import List

import faiss
SEED = 42

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# First part

Here we will carry out the first part of the project

In [4]:
"""
This function allows to download the dataset, it was retrieved directly from the official documentation 
"""
def download_dataset() -> str :
  #### Just some code to print debug information to stdout
  logging.basicConfig(format='%(asctime)s - %(message)s',
                      datefmt='%Y-%m-%d %H:%M:%S',
                      level=logging.INFO,
                      handlers=[LoggingHandler()])
  #### /print debug information to stdout

  #### Download scifact.zip dataset and unzip the dataset
  dataset = "dbpedia-entity"
  url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
  out_dir = os.path.join("./", "datasets")
  return util.download_and_unzip(url, out_dir)



"""
This function allows to load the dataset, it was retrieved directly from the official documentation
"""
def load_dataset(data_path: str) -> tuple:
  #### Provide the data_path where scifact has been downloaded and unzipped
  return GenericDataLoader(data_folder=data_path).load(split="test")



""" 
This function removes the rows with 0 values from the dataset representing 
the relationship between the corpora and the queries.
To do this, it uses the remove_zero_values function 
"""
def remove_unrevelant(d: dict) -> dict:
    return {key: remove_zero_values(d[key]) for key in d}

def remove_zero_values(d: dict) -> dict:
    return {key: value for key, value in d.items() if value != 0}



"""
This function is used to identify texts that are related to a query
"""
def get_all_related_texts(qrels: dict) -> set:
  texts = set()

  for v in qrels.values():
    texts.update(v)

  return texts



"""
This function is used to create the dataset requested by the instruction, 
namely a corpus consisting of all the texts having a relation with the queries as well 
as 100000 other texts

In addition to that, this function encodes directly the texts thanks to our model that we have created beforehand
"""
def get_texts_rels_and_100K_unrel(corpus: dict, texts: set, model: SentenceTransformer, N:int =100000, SEED:int =-1) -> dict:
  if SEED != -1:
    random.seed(SEED)

  dataset = {}

  with tqdm(total=(len(texts) + N)) as pbar:

    for key in texts:
      ele = corpus.pop(key)
      dataset[key] = ele['text']
      pbar.update(1)

    for ele in random.sample(list(corpus.keys()), N):
      dataset[ele] = corpus[ele]['text']
      pbar.update(1)

  emb_values = model.encode(list(dataset.values()), normalize_embeddings=True, show_progress_bar=True)

  dataset = {key: value for key, value in zip(list(dataset.keys()), emb_values)}

  return dataset



"""
This function allows to encode the queries according to the model we have created
"""
def get_emb_queries(queries: dict, model: SentenceTransformer) -> tuple:
  emb_queries = {}

  with tqdm(total=(len(queries))) as pbar:
    for key, value in queries.items():
      emb_queries[key] = model.encode(value, normalize_embeddings=True)
      pbar.update(1)
  
  return emb_queries, list(queries.values())



"""
This function is used to find the average@k score which is calculated in this way:
- First, we do a dot of each query between each text and get the score
- We retrieve the k texts with the highest score
- We make the sum of the successes
- We divide by k to get the average

It uses the get_k_score function to create the k_score and the dot_product function 
to create the dot of each text for a query
"""

def get_k_average(emb_queries: dict, emb_texts: dict , qrels: dict ) -> dict:
  rtn = {}

  with tqdm(total=(len(emb_queries))) as pbar:
    for query, query_emb in emb_queries.items():
      keys = list(emb_texts.keys())
      values = dot_product(list(emb_texts.values()), query_emb)

      dot_texts = {key: value for key, value in zip(keys, values)}

      rtn[query] = (get_k_score(dot_texts, list(qrels[query].keys())))
      pbar.update(1)


  return rtn

def get_k_score(dot_texts: dict , rels: set , k : int =100) -> int:
  dot_lst = {key: value for key, value in sorted(dot_texts.items(), key=lambda item: item[1], reverse=True)[:k]}

  lst = list(dot_lst.keys())

  return len([x for x in rels if x in lst]) / k


def dot_product(text_embendings: List, query_embending) -> List:
  return [sentence_transformers_util.dot_score(query_embending, text_embending) for text_embending in text_embendings]


"""
This function does the same thing as get_k_average using the faiss library
"""
def get_k_average_faiss(emb_queries: dict, emb_texts: dict, qrels: dict, top_k_hits: int, index) -> dict:
  faiss_pred = {}

  with tqdm(total=(len(emb_queries))) as pbar:
    for query, query_emb in emb_queries.items():
      question_embedding = np.expand_dims(query_emb, axis=0)
      distances, corpus_ids = index.search(question_embedding, top_k_hits)

      hits = [{'corpus_id': id, 'score': score} for id, score in zip(corpus_ids[0], distances[0])]
      hits = sorted(hits, key=lambda x: x['score'], reverse=True)

      lst = [list(emb_texts.keys())[hit['corpus_id']] for hit in hits]

      faiss_pred[query] = len([x for x in list(qrels[query].keys()) if x in lst]) / top_k_hits

      pbar.update(1)


  return faiss_pred

### Here we will download and load the datasets

In [5]:
data_path = download_dataset()
corpus, queries, qrels = load_dataset(data_path)

  0%|          | 0/4635922 [00:00<?, ?it/s]

### We clean the dataset of relation between queries and texts.
### It is a dict and here is its architecture

```
key = queries_id
value = dict :
          key = text_id
          value = number_revelant_time
```

In [6]:
qrels = remove_unrevelant(qrels)

### Here we load our model, we chouge him because he his the most efficient according to the documentation

In [7]:
model = SentenceTransformer('msmarco-distilbert-base-tas-b')

### This function retrieves the list of relevant texts

In [8]:
texts = get_all_related_texts(qrels)

### Here we create the dataset requested by the instruction, it contains all the texts released as well as 100000 supplementary. Moreover, we encode the text with the model we have chosen. Here is its architecture :
```
key = text_id
value = text_embended
```

In [9]:
emb_texts = get_texts_rels_and_100K_unrel(corpus.copy(), texts, model, SEED=SEED)

  0%|          | 0/114877 [00:00<?, ?it/s]

Batches:   0%|          | 0/3590 [00:00<?, ?it/s]

### In the same way, we get the list of queries and the encoding with the model.

In [10]:
emb_queries, lst_queries = get_emb_queries(queries, model)

  0%|          | 0/400 [00:00<?, ?it/s]

### Here we retrieve the k_average of each query

This function takes a lot of time to run, ie 55 minutes on google colab, so for the sake of speed, in order to save you time, we have preprocessed this command and save the result in json format

In [11]:
#k_average = get_k_average(emb_queries, emb_texts, qrels)

In [12]:

import json 
with open('json_data.json') as json_file:

  k_average = json.load(json_file)

## As you can see here, here are the different queries with the associated k-average

In [13]:
k_average_labelised = {}

for key, value in k_average.items():
  tmp = queries[key]
  k_average_labelised[tmp] = value

k_average_labelised

{'Szechwan dish food cuisine': 0.14,
 'roman architecture': 0.8,
 'finland car industry manufacturer saab sisu': 0.24,
 'france second world war normandy': 0.41,
 'social network group selection': 0.16,
 'D-Day normandy invasion': 0.59,
 'web ranking scoring algorithm': 0.18,
 'virtual museums': 0.32,
 'Indian food': 0.87,
 'composer museum': 0.07,
 'gallo roman architecture in paris': 0.09,
 'electricity source in France': 0.16,
 'social network API': 0.2,
 'List of films from the surrealist category': 0.08,
 'Einstein Relativity theory': 0.5,
 'summer flowers': 0.64,
 'house concrete wood': 0.07,
 'organic food advantages disadvantages': 0.14,
 'vietnam war movie': 0.38,
 'vietnam food recipes': 0.08,
 'vietnamese food blog': 0.08,
 'vietnam travel national park': 0.37,
 'guitar chord tuning': 0.45,
 'guitar classical flamenco': 0.5,
 'guitar origin Russia': 0.1,
 'guitar origin blues': 0.25,
 'tango culture movies': 0.18,
 'tango culture countries': 0.21,
 'tango music  composers': 

# Second part

Here we use the faiss library to make our recommendations:<br>
We use 1024 clusters because it is an optimized value for the GPU<br>
We use as metric the inner_product because once normalized, it corresponds to the same thing as a cosine similarity.

In [14]:
n_clusters = 1024
embedding_size = len(list(emb_texts.values())[0])
top_k_hits = 100
quantizer = faiss.IndexFlatIP(embedding_size)
index = faiss.IndexIVFFlat(quantizer, embedding_size, n_clusters, faiss.METRIC_INNER_PRODUCT)
index.nprobe = 3

index.train(np.array(list(emb_texts.values())))

index.add(np.array(list(emb_texts.values())))

In [15]:
faiss_pred = get_k_average_faiss(emb_queries, emb_texts, qrels, top_k_hits, index)

  0%|          | 0/400 [00:00<?, ?it/s]

In [16]:
k_average_faiss_labelised = {}

for key, value in faiss_pred.items():
  tmp = queries[key]
  k_average_faiss_labelised[tmp] = value

k_average_faiss_labelised

{'Szechwan dish food cuisine': 0.19,
 'roman architecture': 0.76,
 'finland car industry manufacturer saab sisu': 0.1,
 'france second world war normandy': 0.48,
 'social network group selection': 0.07,
 'D-Day normandy invasion': 0.44,
 'web ranking scoring algorithm': 0.17,
 'virtual museums': 0.28,
 'Indian food': 0.87,
 'composer museum': 0.06,
 'gallo roman architecture in paris': 0.07,
 'electricity source in France': 0.12,
 'social network API': 0.17,
 'List of films from the surrealist category': 0.12,
 'Einstein Relativity theory': 0.42,
 'summer flowers': 0.53,
 'house concrete wood': 0.01,
 'organic food advantages disadvantages': 0.08,
 'vietnam war movie': 0.4,
 'vietnam food recipes': 0.07,
 'vietnamese food blog': 0.05,
 'vietnam travel national park': 0.37,
 'guitar chord tuning': 0.44,
 'guitar classical flamenco': 0.38,
 'guitar origin Russia': 0.04,
 'guitar origin blues': 0.06,
 'tango culture movies': 0.16,
 'tango culture countries': 0.19,
 'tango music  composers